In [1]:
from qem.crystal_analyzer import CrystalAnalyzer
import hyperspy.api as hs   
import scipy.io as sio

file = '/home/zzhang/OneDrive/data/Yang Zhang STO/dynamic pc AXON/adf_avg_25.hspy'
s = hs.load(file)
image = s.data
dx =0.071356
matfile_path = '/home/zzhang/OneDrive/data/Yang Zhang STO/dynamic pc AXON/adf_avg_coords.mat'
matfile = sio.loadmat(matfile_path)
coords = matfile['coords'].astype(float)
crystal_analyzer = CrystalAnalyzer(image, dx, coords[:,:2]/dx, coords[:,2].astype(int)-1,['Y','Al'])

In [ ]:
import numpy as np
cif_file_path = '/home/zzhang/OneDrive/data/Yang Zhang STO/STEM/72104-CaYAlO4-rotated.cif'

crystal_analyzer.read_cif(cif_file_path)


In [ ]:
import numpy as np
crystal_analyzer.origin = np.array([831.26558017, 962.21710935])
crystal_analyzer.a_vector_affine = np.array([ 31.26403339, -36.73158211])
crystal_analyzer.b_vector_affine = np.array([-118.32881689, -100.67459688])
# crystal_analyzer.c = crystal_analyzer.unitcell.cell[2]
atoms = crystal_analyzer.align_unit_cell_to_image(mode='affine')

In [ ]:
%matplotlib qt
crystal_analyzer.get_atomic_columns(reciprocal=False)

In [38]:
crystal_analyzer.plot_unitcell()

In [ ]:
import matplotlib.pyplot as plt
local_dis = crystal_analyzer.atomic_columns.get_local_displacements(50)
dis = crystal_analyzer.atomic_columns.displacements
plt.imshow(image)
plt.scatter(crystal_analyzer.atomic_columns.x, crystal_analyzer.atomic_columns.y, c=np.linalg.norm(dis,axis=1), cmap='coolwarm', s=50)
plt.colorbar()
# plt.clim([0,2])

In [ ]:
import matplotlib.pyplot as plt
shift_origin_adaptive = crystal_analyzer.get_origin_offset(15,15)
shifts = np.array(list(shift_origin_adaptive.values()))
plt.imshow(crystal_analyzer.image)
plt.scatter(shifts[:,0],shifts[:,1],color='y',label='adaptive')

shift_origin_perfect = crystal_analyzer.get_origin_offset(15,15, mode='perfect')
shifts = np.array(list(shift_origin_perfect.values()))
plt.scatter(shifts[:,0],shifts[:,1],color='b',label='perfect')

plt.legend()
plt.title('Unit cell origin locations')

In [ ]:
lattice, lattice_ref = crystal_analyzer.get_lattice_3d(a_limit=20,b_limit=15)
lattice_ref.write('test.xyz')
from ase.visualize import view
view(lattice_ref)

In [32]:
crystal_analyzer.unit_cell.set_tags([0,1,2,3,4,5])

In [ ]:
crystal_analyzer.unit_cell[1].tag

In [ ]:
crystal_analyzer.unit_cell.get_tags()

In [ ]:
view(crystal_analyzer.unit_cell)